<a href="https://colab.research.google.com/github/Herais/NLP_Learning_by_Selective_Data/blob/main/Open.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to project directory
import os
if not os.path.exists('/content/drive/MyDrive/Github'):
  !mkdir Github
path_github   = '/content/drive/MyDrive/Github'
os.chdir(path_github)

# git clone
#!git clone https://github.com/Herais/NLP_Learning_by_Selective_Data.git
path_wd = path_github + '/' + 'NLP_Learning_by_Selective_Data'
os.chdir(path_wd)
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Github/NLP_Learning_by_Selective_Data


In [ ]:
# INSTALL REQUIRED PACKAGES
!pip install -r requirements.txt
%tensorflow_version 1.x

In [ ]:
# LOAD AND EXPLORE DATASETS

# Import Libaries
import numpy as np
import pandas as pd

# Path
path_datasets ='/datasets'

# Load Dataset: Movie Title and Summary
path_dataset_movie = '/Movie_Title_and_Summary'
df_movie = pd.read_json(path_wd + path_datasets + path_dataset_movie + '/train.json', orient='records', lines=True)
df_movie.head(10)

# Load Dataset: Computer Research Literature (CSL) Title and Abstract
path_dataset_csl = '/CSL'
df_csl = pd.read_json(path_wd + path_datasets + path_dataset_csl + '/train.json', orient='records', lines=True)
df_csl.head(10)

,id,title,abst
0,1,保细节的网格刚性变形算法,"提出了一种新的保细节的变形算法,可以使网格模型进行尽量刚性的变形,以减少变形中几何细节的扭曲..."
1,2,一种基于混合模型的实时虚拟人服装动画方法,"实时服装动画生成技术能够为三维虚拟角色实时地生成逼真的服装动态效果,在游戏娱乐、虚拟服装设计..."
2,3,人脸遮挡区域检测与重建,"提出一种基于模糊主分量分析技术(FPCA)的人脸遮挡检测与去除方法.首先,有遮挡人脸被投影到..."
3,4,一种基于奇异值分解的图像匹配算法,图像匹配技术在计算机视觉、遥感和医学图像分析等领域有着广泛的应用背景.针对传统的相关匹配算法...
4,5,片相似性各项异性扩散图像去噪,提出了一种基于片相似性的各项异性扩散图像去噪方法.传统的各项异性图像去噪方法都是基于单个像素...
5,6,基于软硬数据的多点地质统计法在图像统计信息重构中的应用研究,"仅使用硬数据或无条件数据时,图像统计信息的重构会比较困难而且精度不高.如果在重构过程中加入软..."
6,7,一种基于自适应区域分割的地形模型简化方法,"鉴于统一误差计算模型简化方法自适应性差的问题,提出了一种根据地势特征自适应分割地形区域从而有..."
7,8,基于样例的交互式三维动画的生成,"在基于草图的三维动画复制的基础上,提出了基于样例的交互式三维动画生成方法.在保留源动画基本风..."
8,9,一种挖掘压缩序列模式的有效算法,"从序列数据库中挖掘频繁序列模式是数据挖掘领域的一个中心研究主题,而且该领域已经提出和研究了各..."
9,10,基于KL距离的非平衡数据半监督学习算法,"在实际应用中,由于各种原因时常无法直接获得已标识反例,导致传统分类方法暂时失灵,因此,基于正..."


In [ ]:
# SET MODEL PARAMETERS

# Set Model Parameter
epochs = 10
batch_size = 8
maxlen = 256
topk = 1

# Download pretrained BERT model if necesary
path_pretrained_model = '/pretrained_model'
path_bert_chinese = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12'

if not os.path.exists(path_bert_chinese):
 # Download Model
  !wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip -P './pretrained_model/'
  %cd pretrained_model
  !unzip chinese_L-12_H-768_A-12.zip
  !rm -f chinese_L-12_H-768_A-12.zip
  os.chdir(path_wd)
  !pwd

# Set Pretrained Model Parameters (BERT)
config_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/vocab.txt'


# Load Vocab
from bert4keras.tokenizers import Tokenizer, load_vocab

token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)

In [ ]:

# TRAIN MODELS
# Skip to LOAD TRAINED MODELS to use existing models for prediction

# Load Libraries
from __future__ import print_function
import numpy as np
import json
from tqdm import tqdm
from bert4keras.backend import keras, K
from bert4keras.layers import Loss
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder
#from tensorflow.keras.models import Model
from keras.models import Model
from rouge import Rouge  # pip install rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import model as md
import utils

# Load train data
path_train = path_wd + path_datasets + '/CSL/train.json'
path_dev = path_wd + path_datasets + '/CSL/dev.json'
train_data = md.load_data(path_train)
valid_data = md.load_data(path_dev)
# Could use pandas, but pandas has limitation on cell string length

# Initialize Model with Pretrained BERT
token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)
model = md.build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,  # 只保留keep_tokens中的字，精简原字
)

autotitle = AutoTitle(start_id=None, end_id=tokenizer._token_end_id, maxlen=32)
evaluator = Evaluator()
output = md.CrossEntropy(2)(model.inputs + model.outputs)
model = md.Model(model.inputs, output)
train_generator = md.data_generator(train_data, batch_size)
model.compile(optimizer=md.Adam(1e-5))
#model.summary()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    callbacks=[evaluator]
)

# Save Trained Model
path_models = '/models'
utils.save_model_with_dt(path_wd + path_models, model, 'new_train_test')

NameError: ignored

In [ ]:
# LOAD TRAINED MODELS
from tensorflow import keras
from bert4keras.models import build_transformer_model

path_models = '/models'

# the Star Chaser Model trained with movie dataset only
model_StarChaser = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,  # include only tokens in keep tokens
)
model_StarChaser.load_weights(path_wd + path_models + '/best_model_StarChaser.weights')

# the Scholar Model trained with Computer Science Literature only
model_Scholar = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,   # include only tokens in keep tokens
)
model_Scholar.load_weights(path_wd + path_models + '/best_model_Scholar.weights')

# the Know-it-all Model Trained with both movie and cscl dataset
model_Knowitall = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,   # include only tokens in keep tokens
) 
model_Knowitall.load_weights(path_wd + path_models + '/best_model_Knowitall.weights')

In [ ]:
# PREDICTION

# Load Library
import json
from model import AutoTitle

path_predictions = '/predictions'
path_test = path_wd + path_datasets + path_dataset_csl + '/test.json'
path_prediction = path_wd + path_predictions + 'Model_StarChaser_predict_CSLTitle.json'
test_data = []

autotitle = AutoTitle(start_id=None, end_id=tokenizer._token_end_id, maxlen=32)

with open(path_test, encoding='utf-8') as f:
    for l in f:
        test_data.append(json.loads(l.strip()))

with open(path_prediction ,'w') as f:
    for data in test_data:
        pred_title = ''.join(autotitle.generate(data["abst"], topk)).lower()
        output = {}
        output["id"] = data["id"]
        output["title"] = pred_title
        f.write(json.dumps(output, ensure_ascii = False))
        f.write('\n')

In [ ]:
# RESULT ANALYSIS